In [1]:
# Dependencies
import pandas as pd
from datetime import date
import sqlite3
import csv

In [2]:
hist_la_crime = pd.read_csv('../../LA_Crime_Data/Crime_Data_from_2010_to_Present.csv')
hist_la_crime = hist_la_crime.rename(columns={'DR Number':'dr_no',
                                              'Date Reported':'date_rptd',
                                              'Date Occurred':'date_occ',
                                              'Time Occurred':'time_occ',
                                              'Area ID':'area_id',
                                              'Area Name':'area_name',
                                              'Reporting District':'rpt_dist_no',
                                              'Crime Code':'crm_cd',
                                              'Crime Code Description':'crm_cd_desc',
                                              'MO Codes':'mocodes',
                                              'Victim Age':'vict_age',
                                              'Victim Sex':'vict_sex',
                                              'Victim Descent':'vict_descent',
                                              'Premise Code':'premis_cd',
                                              'Premise Description':'premis_desc',
                                              'Weapon Used Code':'weapon_used_cd',
                                              'Weapon Description':'weapon_desc',
                                              'Status Code':'status',
                                              'Status Description':'status_desc',
                                              'Crime Code 1':'crm_cd_1',
                                              'Crime Code 2':'crm_cd_2',
                                              'Crime Code 3':'crm_cd_3',
                                              'Crime Code 4':'crm_cd_4',
                                              'Address':'location',
                                              'Cross Street':'cross_street',
                                              'Location ':'location_1'})


hist_la_crime.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area_id', 'area_name',
       'rpt_dist_no', 'crm_cd', 'crm_cd_desc', 'mocodes', 'vict_age',
       'vict_sex', 'vict_descent', 'premis_cd', 'premis_desc',
       'weapon_used_cd', 'weapon_desc', 'status', 'status_desc', 'crm_cd_1',
       'crm_cd_2', 'crm_cd_3', 'crm_cd_4', 'location', 'cross_street',
       'location_1'],
      dtype='object')

In [3]:
# Create a Pandas DataFrame with the results
hist_la_crime = hist_la_crime.set_index(['dr_no'])
# hist_la_crime = hist_la_crime.sort_values(by='date_occ', ascending=False)

hist_la_crime.head()

,date_rptd,date_occ,time_occ,area_id,area_name,rpt_dist_no,crm_cd,crm_cd_desc,mocodes,vict_age,...,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,location_1
dr_no,,,,,,,,,,,,,,,,,,,,,
1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,30.0,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,12.0,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,84.0,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [4]:
# Set column datatypes and drop some columns
hist_la_crime['area_id'] = hist_la_crime['area_id'].apply(pd.to_numeric)
hist_la_crime['crm_cd'] = hist_la_crime['crm_cd'].apply(pd.to_numeric)
hist_la_crime['crm_cd_1'] = hist_la_crime['crm_cd_1'].apply(pd.to_numeric)
hist_la_crime['crm_cd_2'] = hist_la_crime['crm_cd_2'].apply(pd.to_numeric)
hist_la_crime['crm_cd_3'] = hist_la_crime['crm_cd_3'].apply(pd.to_numeric)
hist_la_crime['date_occ'] = pd.to_datetime(hist_la_crime['date_occ'], infer_datetime_format=True)
hist_la_crime['date_rptd'] = pd.to_datetime(hist_la_crime['date_rptd'], infer_datetime_format=True)
hist_la_crime[['latitude','longitude']] = hist_la_crime['location_1'].str.strip('()').str.split(', ', expand=True).rename(columns={0:'latitude', 1:'longitude'})
hist_la_crime['latitude'] = hist_la_crime['latitude'].apply(pd.to_numeric)
hist_la_crime['longitude'] = hist_la_crime['longitude'].apply(pd.to_numeric)
hist_la_crime = hist_la_crime.drop(columns=['location_1'])
hist_la_crime['premis_cd'] = hist_la_crime['premis_cd'].apply(pd.to_numeric)
hist_la_crime['rpt_dist_no'] = hist_la_crime['rpt_dist_no'].apply(pd.to_numeric)
hist_la_crime['time_occ'] = hist_la_crime['time_occ'].apply(str).apply(lambda x: x.zfill(4))
hist_la_crime['hour_occ'] = hist_la_crime['time_occ'].apply(str).str[:2].apply(pd.to_numeric)
hist_la_crime['minute_occ'] = hist_la_crime['time_occ'].apply(str).str[2:].apply(pd.to_numeric)
hist_la_crime = hist_la_crime.drop(columns=['time_occ'])
hist_la_crime['vict_age'] = hist_la_crime['vict_age'].apply(pd.to_numeric)
hist_la_crime['weapon_used_cd'] = hist_la_crime['weapon_used_cd'].apply(pd.to_numeric)

hist_la_crime.dtypes

date_rptd         datetime64[ns]
date_occ          datetime64[ns]
area_id                    int64
area_name                 object
rpt_dist_no                int64
crm_cd                     int64
crm_cd_desc               object
mocodes                   object
vict_age                 float64
vict_sex                  object
vict_descent              object
premis_cd                float64
premis_desc               object
weapon_used_cd           float64
weapon_desc               object
status                    object
status_desc               object
crm_cd_1                 float64
crm_cd_2                 float64
crm_cd_3                 float64
crm_cd_4                 float64
location                  object
cross_street              object
latitude                 float64
longitude                float64
hour_occ                   int64
minute_occ                 int64
dtype: object

In [5]:
hist_la_crime.head()

,date_rptd,date_occ,area_id,area_name,rpt_dist_no,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,...,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,latitude,longitude,hour_occ,minute_occ
dr_no,,,,,,,,,,,,,,,,,,,,,
1208575,2013-03-14,2013-03-11,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,30.0,F,...,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,33.9829,-118.3338,18,0
102005556,2010-01-25,2010-01-22,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,NaN,NaN,...,510.0,NaN,NaN,NaN,VAN NESS,15TH,34.0454,-118.3157,23,0
418,2013-03-19,2013-03-18,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,12.0,NaN,...,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,33.9420,-118.2717,20,30
101822289,2010-11-11,2010-11-10,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,NaN,NaN,...,510.0,NaN,NaN,NaN,88TH,WALL,33.9572,-118.2717,18,0
42104479,2014-01-11,2014-01-04,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,84.0,M,...,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,34.2009,-118.6369,23,0


In [7]:
# Create sqlalchemy engine
from sqlalchemy import create_engine

conn = sqlite3.connect("../db/la_crime.db")
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS hist_la_crime;''')
c.execute('''CREATE TABLE hist_la_crime
            (dr_no INTEGER PRIMARY KEY ASC,
             area_id INTEGER,
             area_name VARCHAR(64),
             crm_cd INTEGER,
             crm_cd_1 INTEGER,
             crm_cd_2 INTEGER,
             crm_cd_3 INTEGER,
             crm_cd_4 INTEGER,
             crm_cd_desc VARCHAR(64),
             cross_street VARCHAR(64),
             date_occ DATE,
             date_rptd DATE,
             location VARCHAR(64),
             longitude FLOAT,
             latitude FLOAT,
             mocodes TEXT,
             premis_cd INTEGER,
             premis_desc VARCHAR(64),
             rpt_dist_no INTEGER,
             status VARCHAR(2),
             status_desc VARCHAR(64),
             hour_occ INTEGER,
             minute_occ INTEGER,
             vict_age INTEGER,
             vict_descent VARCHAR(64),
             vict_sex VARCHAR(2),
             weapon_desc VARCHAR(64),
             weapon_used_cd INTEGER,
             loc_type VARCHAR(64))
             ''')
conn.commit()
conn.close()

In [9]:
# Create original table
engine = create_engine('sqlite:///../db/la_crime.db')
hist_la_crime.to_sql('hist_la_crime', engine, if_exists='append', index=True, )